<a href="https://colab.research.google.com/github/adichat08/CNN-for-Bird-s-Eye-Human-Detection/blob/main/Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Models

In [ ]:
# imports specific to the model construction
import cv2
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import keras
from keras.models import Sequential, Model,load_model
from tensorflow.keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
from google.colab.patches import cv2_imshow
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from keras.preprocessing import image
from keras.initializers import glorot_uniform

In [ ]:
# This is code to check GPU usage


# memory footprint support libraries/code
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize

# import psutil
# import humanize
# import os
# import GPUtil as GPU

# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
# def printm():
#     process = psutil.Process(os.getpid())
#     print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
#     print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm()

Implementation of basic VGG 16 architecture

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# Defining the VGG-16 model architecture
vgg_16 = keras.Sequential([
    # Convolutional Block 1
    # The first convolutional layer is the input layer and expects images of size 150x150 pixels with 3 color channels (RGB).
    # 64 filters are chosen to capture a variety of low-level features in the input images.
    # A 3x3 kernel size is commonly used in CNNs to capture spatial information.
    # 'Same' padding is chosen so that the spatial dimensions of the output feature maps match the dimensions of the input images.
    # ReLU activation is used as non-linearity, as opposed to other activation functions such as sigmoid. ReLU is used more commonly as it better addresses the vanising gradient problem and results in more sparse activation.
    layers.Conv2D(input_shape=(150, 150, 3), filters=64, kernel_size=(3, 3), padding="same", activation="relu"),
    layers.Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu"),
    # Max pooling is used to downsample the feature maps, reducing spatial dimensions and as a result, computational complexity.
    # A pool size of 2x2 and a stride of 2x2 are chosen to halve the spatial dimensions after each max-pooling layer.
    layers.MaxPool2D(pool_size=(2,2),strides=(2,2)),

    # Convolutional Block 2
    # The number of filters is increased so that the model can now capture higher-level features (it can learn more complex patterns)
    # The same padding and ReLU activation function are used for consistency.
    layers.Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"),
    layers.Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"),
    # Max pooling is still downsampling feature maps, while also preserving important features that the model identified.
    layers.MaxPool2D(pool_size=(2,2),strides=(2,2)),

    # Convolutional Block 3
    # Increasing the number of filters to capture even more abstract features.
    layers.Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"),
    layers.Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"),
    layers.Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"),
    layers.MaxPool2D(pool_size=(2,2),strides=(2,2)),


    # Convolutional Block 4
    # Continuing the trend of increasing filters for deeper representation.
    layers.Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"),
    layers.Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"),
    layers.Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"),
    layers.MaxPool2D(pool_size=(2,2),strides=(2,2)),

    # Convolutional Block 5
    # Keeping the number of filters consistent for the final convolutional layers.
    layers.Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"),
    layers.Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"),
    layers.Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"),
    layers.MaxPool2D(pool_size=(2,2),strides=(2,2)),

    # Fully Connected Layers
    # Flattening the 3D convolutional outputs into a 1D vector for input to the fully connected layers.
    # Dense layers with 4096 units are chosen to preserve the high-level feature representations acquired in the deeper layers.
    # ReLU activation used for the aformentioned reasons.

    layers.Flatten(),
    layers.Dense(units=4096, activation="relu"),
    layers.Dense(units=4096, activation="relu"),

    # Output Layer
    # The output layer consists of 2 units, representing the two classes (human target identified or not).
    # Softmax activation is chosen for the output layer to produce class probabilities.
    layers.Dense(units=2, activation="softmax")
])


In [ ]:
from tensorflow.keras.optimizers import Adam
# Adam is a common optimizer and a learning rate of 0.001 is generally shown to be stable and work well for a variety of tasks. This is an intial hyperparameter choice.
opt = Adam(learning_rate=0.001)

In [ ]:
vgg_16.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [ ]:
vgg_16.summary()

Implementation of modified ResNet50 architecture

In [ ]:
# Below is an explanation for the general structure of the model:

# Convolutional Block and Identity Block Structure:
# The ResNet architecture is based on the concept of residual learning, where the main path (through convolutional layers) is augmented with a shortcut connection (skipping layers).
# Both convolutional_block and identity_block functions are fundamental building blocks of ResNet, each consisting of multiple convolutional layers, followed by batch normalization and ReLU activation.

# Number of Filters:
# The number of filters in each convolutional layer is carefully chosen to balance model complexity and representational capacity.
# For example, in the convolutional_block, the number of filters gradually increases from F1 to F3, capturing more abstract features as the network progresses.

# Kernel Size and Padding:
# Kernel size of 1x1, 3x3, and occasionally 7x7 is used in different layers to capture different levels of spatial information.
# Same padding is used in most convolutional layers to preserve spatial dimensions.

# Striding:
# Striding is employed in the convolutional_block to reduce spatial dimensions (e.g., from 56x56 to 28x28) when transitioning between different stages of the network.

# Pooling:
# Average pooling is applied at the end of the network to aggregate spatial information and reduce spatial dimensions before the final output layer.

def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block in ResNet50.

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- list of integers, defining the number of filters in the CONV layers
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network

    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    # Define name basis for the layers
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X


    # First component of main path
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    # Add shortcut value to main path and pass it through a ReLU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

def convolutional_block(X, f, filters, stage, block, s=2):
    """
    Implementation of the convolutional block in ResNet50.

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used

    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    # Define name basis for the layers
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    # Shortcut path
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    # Add shortcut value to main path and pass it through a ReLU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

def ResNet50(input_shape=(224, 224, 3)):
    """
    Implementation of the ResNet50 architecture.

    Arguments:
    input_shape -- shape of the images of the dataset

    Returns:
    model -- a Model() instance in Keras
    """
    # Define input with zero padding
    X_input = Input(input_shape)
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # Average pooling layer
    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)

    # Create model
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model


In [ ]:
base_model = ResNet50(input_shape=(150, 150, 3))

Transfer learning implementation attempt

In [ ]:
headModel = base_model.output
headModel = Flatten()(headModel)
headModel=Dense(256, activation='relu', name='fc1',kernel_initializer=glorot_uniform(seed=0))(headModel)
headModel=Dense(128, activation='relu', name='fc2',kernel_initializer=glorot_uniform(seed=0))(headModel)
headModel = Dense( 1,activation='sigmoid', name='fc3',kernel_initializer=glorot_uniform(seed=0))(headModel)

In [ ]:
model = Model(inputs=base_model.input, outputs=headModel)

In [ ]:
model.summary()

In [ ]:
base_model.load_weights('/content/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
for layer in model.layers:
    print(layer, layer.trainable)

In [ ]:
es=EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=20)

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(expanded_dataset, expanded_y, epochs=20, batch_size = 8, validation_split = 0.2, verbose = 1, callbacks = [es])

Below are tests for basic CNNs

In [ ]:
# Testing a CNN

cnn = keras.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

In [ ]:
# Testing a CNN

cnn_2 = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)),

    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),

    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),

    layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(2, activation='softmax')
])

In [ ]:
# Testing a CNN

cnn_3 = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(125, 125, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

In [ ]:
# Testing a CNN

cnn_3_1 = models.Sequential([
    layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(125, 125, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(2, activation='softmax')
])

In [ ]:
# Testing a CNN

cnn_3_2 = models.Sequential([
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=(125, 125, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(2, activation='softmax')
])

In [ ]:
# Testing a CNN

cnn_4 = models.Sequential([
    layers.Conv2D(filters=128, kernel_size=(7, 7), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(filters=64, kernel_size=(5, 5), activation='relu'),
    #layers.MaxPooling2D((2, 2)),

    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

In [ ]:
# Testing a CNN

cnn_5 = models.Sequential([
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

In [ ]:

cnn_4 = models.Sequential([
    layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    #layers.MaxPooling2D((2, 2)),

    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'),
    #layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

In [ ]:
cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
cnn_2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
cnn_3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
cnn_3_1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
cnn_3_2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
cnn_4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
cnn_5.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
vgg_16.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Fitting models

In [ ]:
history_c3_1_4 = cnn_3.fit(expanded_X_train, expanded_y_train, epochs=200, batch_size = 8, validation_split = 0.2)

In [ ]:
history_c3_1_3 = cnn_3.fit(expanded_X_train, expanded_y_train, epochs=500, batch_size = 8, validation_split = 0.2)

In [ ]:
history_c3_1_3 = cnn_3.fit(expanded_X_train, expanded_y_train, epochs=15, batch_size = 8, validation_split = 0.2)

In [ ]:
history_c3_1_4 = cnn_3.fit(FIRE_X_train, FIRE_y_train, epochs=15, batch_size = 8, validation_split = 0.2)

In [ ]:
history_c3_1_2 = cnn_3.fit(expanded_X_train, expanded_y_train, epochs=50, batch_size = 4, validation_split = 0.2)

In [ ]:
history_c3_1_1 = cnn_3.fit(expanded_X_train, expanded_y_train, epochs=50, batch_size = 16, validation_split = 0.2)

In [ ]:
history_c3_1_1 = cnn_3.fit(expanded_X_train, expanded_y_train, epochs=50, batch_size = 1, validation_split = 0.2)

In [ ]:
history_c3_2 = cnn_3_2.fit(expanded_X_train, expanded_y_train, epochs=50, batch_size = 8, validation_split = 0.2)

In [ ]:
history_c3_1 = cnn_3_1.fit(expanded_X_train, expanded_y_train, epochs=50, batch_size = 8, validation_split = 0.2)

In [ ]:
history_c3_1 = cnn_3.fit(expanded_X_train, expanded_y_train, epochs=50, batch_size = 8, validation_split = 0.2)

In [ ]:
history_c3_1 = cnn_3.fit(expanded_X_train, expanded_y_train, epochs=50, batch_size = 4, validation_split = 0.2)

In [ ]:
history_104 = cnn_2.fit(X_train_full, y_train_full, epochs=500, batch_size = 8, validation_split = 0.2)

In [ ]:
history_104 = cnn_2.fit(expanded_dataset, expanded_y, epochs=20, batch_size = 4, validation_split = 0.2)

In [ ]:
history_104 = cnn_2.fit(expanded_dataset, expanded_y, epochs=20, batch_size = 8, validation_split = 0.3)

In [ ]:
history_104 = cnn_2.fit(expanded_dataset, expanded_y, epochs=20, batch_size = 8, validation_split = 0.2)

In [ ]:
history_104 = cnn_3.fit(expanded_dataset, expanded_y, epochs=20, batch_size = 8, validation_split = 0.2)

In [ ]:
history_104 = cnn_4.fit(expanded_dataset, expanded_y, epochs=20, batch_size = 8, validation_split = 0.2)

In [ ]:
history_104 = cnn_4.fit(expanded_dataset, expanded_y, epochs=100, batch_size = 8, validation_split = 0.2)

In [ ]:
history_104 = cnn_4.fit(expanded_dataset, expanded_y, epochs=50, batch_size = 8, validation_split = 0.2)

Displaying graphs

In [ ]:
# summarize history for accuracy
plt.figure(figsize=(15,10))
plt.plot(history_104.history['accuracy'])
plt.plot(history_104.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.grid()
plt.show()
# summarize history for loss
plt.figure(figsize=(15,10))
plt.plot(history_104.history['loss'])
plt.plot(history_104.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.grid()
plt.show()

In [ ]:
# summarize history for accuracy
plt.figure(figsize=(15,10))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.grid()
plt.show()
# summarize history for loss
plt.figure(figsize=(15,10))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.grid()
plt.show()

In [ ]:
# summarize history for accuracy
plt.figure(figsize=(15,10))
plt.plot(history_2.history['accuracy'])
plt.plot(history_2.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.grid()
plt.show()
# summarize history for loss
plt.figure(figsize=(15,10))
plt.plot(history_2.history['loss'])
plt.plot(history_2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.grid()
plt.show()

In [ ]:
# summarize history for accuracy
plt.figure(figsize=(15,10))
plt.plot(history_3.history['accuracy'])
plt.plot(history_3.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.grid()
plt.show()
# summarize history for loss
plt.figure(figsize=(15,10))
plt.plot(history_3.history['loss'])
plt.plot(history_3.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.grid()
plt.show()

In [ ]:
# summarize history for accuracy
plt.figure(figsize=(15,10))
plt.plot(history_4.history['accuracy'])
plt.plot(history_4.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.grid()
plt.show()
# summarize history for loss
plt.figure(figsize=(15,10))
plt.plot(history_4.history['loss'])
plt.plot(history_4.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.grid()
plt.show()

In [ ]:
# summarize history for accuracy
plt.figure(figsize=(15,10))
plt.plot(history_5.history['accuracy'])
plt.plot(history_5.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.grid()
plt.show()
# summarize history for loss
plt.figure(figsize=(15,10))
plt.plot(history_5.history['loss'])
plt.plot(history_5.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.grid()
plt.show()

In [ ]:
# summarize history for accuracy
plt.figure(figsize=(15,10))
plt.plot(history_11.history['accuracy'])
plt.plot(history_11.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.grid()
plt.show()
# summarize history for loss
plt.figure(figsize=(15,10))
plt.plot(history_11.history['loss'])
plt.plot(history_11.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.grid()
plt.show()

Checking results

In [ ]:
cnn_3.evaluate(X_test,y_test)

In [ ]:
cnn_3_1.evaluate(X_test,y_test)

In [ ]:
cnn_3.evaluate(X_test ,y_test)

In [ ]:
cnn_3.evaluate(X_test ,y_test)

In [ ]:
cnn_3.evaluate(X_test ,y_test)

In [ ]:
cnn_3.evaluate(X_test ,y_test)

In [ ]:
cnn_3.evaluate(X_test ,y_test)

In [ ]:
cnn_3.evaluate(X_test ,y_test)

In [ ]:
cnn_3.evaluate(X_test ,y_test)

In [ ]:
cnn_4.evaluate(X_test ,y_test)

In [ ]:
cnn_4.evaluate(X_test ,y_test)

In [ ]:
cnn_4.evaluate(X_test ,y_test)

In [ ]:
cnn_2.evaluate(X_test ,y_test)

In [ ]:
model.evaluate(X_test ,y_test)

In [ ]:
vgg_16.evaluate(X_test ,y_test)

In [ ]:
cnn_3.evaluate(expanded_X_test ,expanded_y_test)

In [ ]:
cnn_3.evaluate(expanded_X_test ,expanded_y_test)

In [ ]:
cnn_3_1.evaluate(expanded_X_test ,expanded_y_test)

In [ ]:
cnn_3_2.evaluate(expanded_X_test ,expanded_y_test)

In [ ]:
cnn_3.evaluate(expanded_X_test ,expanded_y_test)

In [ ]:
cnn_3.evaluate(expanded_X_test ,expanded_y_test)

In [ ]:
cnn_3.evaluate(expanded_X_test ,expanded_y_test)

In [ ]:
cnn_3.evaluate(expanded_X_test ,expanded_y_test)

In [ ]:
cnn_3.evaluate(expanded_X_test ,expanded_y_test)

In [ ]:
cnn_3.evaluate(FIRE_dataset, humans_in_fire_y)

In [ ]:
cnn_3.evaluate(FIRE_X_test, FIRE_y_test)

In [ ]:
y_pred_cnn = cnn_3.predict(X_test)

In [ ]:
y_pred_cnn_1 = cnn_3.predict(FIRE_X_test)

In [ ]:
y_classes_cnn = [np.argmax(element) for element in y_pred_cnn_1]

In [ ]:
print(list(y_test[:10]))
print(y_classes_cnn[:10])

In [ ]:
print("Classification Report: \n", classification_report(FIRE_y_test, y_classes_cnn))

In [ ]:
print("Classification Report: \n", classification_report(y_test, y_classes_cnn))

In [ ]:
print("Classification Report: \n", classification_report(y_test, y_classes_cnn))

In [ ]:
print("Classification Report: \n", classification_report(y_test, y_classes_cnn))

In [ ]:
print("Classification Report: \n", classification_report(y_test, y_classes_cnn))